In [ ]:
import os
import sys
import rootpy.ROOT as ROOT
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from rootpy.plotting import *
import rootpy.plotting.root2matplotlib as rplt
from rootpy.interactive import wait
import seaborn

In [2]:
# mjj plots
palette = seaborn.cubehelix_palette(n_colors=7, start=0, rot=0.4, gamma=1.0, hue=0.8, light=0.85, dark=0.15, reverse=False, as_cmap=False)
colors = {"CSVT":palette[0], "CSVTM":palette[1], "CSVTL":palette[2], "CSVM":palette[3], "CSVML":palette[4], "CSVL":palette[5], "NoBTag":palette[6]}


In [ ]:
print "Loading histograms"
analyses = []
masses = {}
hist_mjj = {}
hist_mjj_eff = {}
for sr in ["NoTrigger_eta1p7", "NoTrigger_eta2p2"]: 
    print "SR {}".format(sr)
    for wp in ["CSVT", "CSVM", "CSVL", "CSVTM", "CSVTL", "CSVML", "NoBTag"]:
        if wp == "NoBTag":
            analysis = sr
        else:
            analysis = sr + "_" + wp
        analyses.append(analysis)
        if sr == "NoTrigger_eta1p7":
            masses[analysis] = [350, 400, 500, 600]
        else:
            masses[analysis] = [600, 750, 900, 1200]
        for mass in masses[analysis]:
            f = ROOT.TFile("BHistograms_{}_GluGluSpin0ToBBbar_M_{}_TuneCUEP8M1_8TeV_pythia8_FULLSIM.root".format(analysis, mass), "READ")
            if analysis in hist_mjj:
                hist_mjj[analysis].Add(f.Get("BHistograms/h_pfjet_mjj").Rebin(10))
            else:
                hist_mjj[analysis] = f.Get("BHistograms/h_pfjet_mjj").Clone()
                hist_mjj[analysis].Rebin(10)
                hist_mjj[analysis].SetName("h_pfjet_mjj_" + analysis)
                hist_mjj[analysis].SetDirectory(0)
            f.Close()
    print "\tMaking plot"
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7,7), dpi=100, sharex=True)
    for wp in ["CSVT", "CSVTM", "CSVTL", "CSVM", "CSVML", "CSVL", "NoBTag"]:
        if wp == "NoBTag":
            analysis = sr
        else:
            analysis = sr + "_" + wp
        hist_mjj[analysis].markercolor = colors[wp]
        hist_mjj[analysis].markerstyle = 20
        hist_mjj[analysis].linecolor = colors[wp]
        rplt.errorbar(hist_mjj[analysis], axes=ax1)
        if not wp == "NoBTag":
            hist_mjj_eff[analysis] = hist_mjj[analysis].Clone()
            hist_mjj_eff[analysis].Divide(hist_mjj[analysis], hist_mjj[sr], 1, 1, "B")
            hist_mjj_eff[analysis].markercolor = colors[wp]
            hist_mjj_eff[analysis].markerstyle = 21
            hist_mjj_eff[analysis].linecolor = colors[wp]
            rplt.errorbar(hist_mjj_eff[analysis], axes=ax2)
    ax1.set_xlim(0., 2000.)
    ax2.set_xlim(0., 2000.)
    ax2.set_ylim(0., 0.7)


In [35]:
# Single jet efficiency
pt_colors = {}
pt_colors["CSVM"] = "red"
pt_colors["CSVT"] = "blue"
pt_colors["CSVL"] = "green"
pt_colors["NoBTag"] = "black"
for sr in ["NoTrigger_eta1p7", "NoTrigger_eta2p2"]: 
    for jet_index in [0, 1]:
        hist_jet_pt_eta = {}
        if sr == "NoTrigger_eta1p7":
            masses = [400, 500, 600]
        else:
            masses = [600, 750, 900, 1200]
        for mass in masses:
            f = ROOT.TFile("BHistograms_{}_GluGluSpin0ToBBbar_M_{}_TuneCUEP8M1_8TeV_pythia8_FULLSIM.root".format(sr, mass), "READ")        
            for wp in ["CSVT", "CSVM", "CSVL"]:
                if not wp in hist_jet_pt_eta:
                    hist_jet_pt_eta[wp] = f.Get("BHistograms/h_pfjet_jet{}_pt_eta_{}".format(jet_index, wp))
                    hist_jet_pt_eta[wp].SetName("h_pfjet_jet{}_pt_eta_{}_{}".format(jet_index, wp, sr))
                    hist_jet_pt_eta[wp].SetDirectory(0)
                else:
                    hist_jet_pt_eta[wp].Add(f.Get("BHistograms/h_pfjet_jet{}_pt_eta_{}".format(jet_index, wp)))
            if not "NoBTag" in hist_jet_pt_eta:
                hist_jet_pt_eta["NoBTag"] = f.Get("BHistograms/h_pfjet_jet{}_pt_eta".format(jet_index))
                hist_jet_pt_eta["NoBTag"].SetName("h_pfjet_jet{}_pt_eta_{}_{}".format(jet_index, "NoBTag", sr))
                hist_jet_pt_eta["NoBTag"].SetDirectory(0)
            else:
                hist_jet_pt_eta["NoBTag"].Add(f.Get("BHistograms/h_pfjet_jet{}_pt_eta".format(jet_index)))
            f.Close()
        fig_jet, (ax1_jet, ax2_jet) = plt.subplots(2, 1, figsize=(7,7), dpi=100, sharex=True)
        hist_jet_pt = {}
        for wp, hist in hist_jet_pt_eta.iteritems():
            hist_jet_pt[wp] = ROOT.TH1D(hist.ProjectionX())
            hist_jet_pt[wp].SetName("hist_jet_pt_{}".format(wp))
            hist_jet_pt[wp].SetDirectory(0)
            hist_jet_pt[wp].markercolor = pt_colors[wp]
            hist_jet_pt[wp].markersize = 1
            hist_jet_pt[wp].markerstyle = 20
            hist_jet_pt[wp].linecolor = pt_colors[wp]
            #hist_jet_pt[wp].markercolor = colors[wp]
            #hist_jet_pt[wp].markerstyle = 20
            #hist_jet_pt[wp].linecolor = colors[wp]
            print hist_jet_pt[wp]
            rplt.errorbar(hist_jet_pt[wp], axes=ax1_jet, label=wp)
        ax1_jet.legend()
        
        hist_jet_pt_eff = {}
        for wp in ["CSVT", "CSVM", "CSVL"]:
            hist_jet_pt_eff[wp] = hist_jet_pt[wp].Clone()
            hist_jet_pt_eff[wp].Divide(hist_jet_pt[wp], hist_jet_pt["NoBTag"], 1, 1, "B")
            rplt.errorbar(hist_jet_pt_eff[wp], axes=ax2_jet, label=wp)

        # Some other trends
        official_btag_x = np.array([35.01322097249651,45.107231123146036,55.16876698595725,65.10668402172008,75.10776089386712,90.08848845940459,110.18346070423463,140.11633345876524,185.26200781836886,234.82713778803898,290.01916979715446,360.5172086192683,449.60976166041274,584.8993623302696])
        official_btag_y = np.array([0.5960687119365385,0.6188028404412305,0.6640411880284045,0.679772862535363,0.7076261444516009,0.7216359441369021,0.730458106752978,0.7410437235543019,0.7221789217085043,0.7015482198325412,0.6739615350122701,0.6359751176043765,0.5858524177177007,0.547900242138647])
        hbb_btag_x = np.array([53.81984036488028,72.97605473204108,83.01026225769671,93.04446978335234,102.16647662485751,112.20068415051314,121.32269099201825,131.35689851767393,140.478905359179,150.5131128848347,169.66932725199547,188.8255416191563,207.98175598631707,228.05017103762833,247.2063854047891,266.3625997719499,295.5530216647663,317.4458380843786,353.93386545039914,394.98289623717227,440.59293044469786,490.76396807297607,547.320410490308,610.2622576966934,680.5017103762829,758.9509692132269])
        hbb_btag_y = np.array([0.539016393442623,0.5763934426229508,0.5911475409836066,0.5724590163934427,0.5527868852459017,0.5685245901639344,0.5557377049180328,0.5832786885245902,0.5340983606557377,0.49278688524590164,0.46229508196721314,0.4121311475409836,0.4278688524590164,0.401311475409836,0.380655737704918,0.3472131147540984,0.32754098360655737,0.28918032786885245,0.27147540983606555,0.2104918032786885,0.19573770491803277,0.16524590163934427,0.13081967213114754,0.11213114754098352,0.09245901639344267,0.08163934426229502])
        csvl_btag_x = np.array([47.41980474198044 ,150.6276150627615 ,251.04602510460245 ,348.67503486750354 ,451.8828451882846 ,546.7224546722455 ,649.9302649930264 ,753.1380753138076 ,847.9776847977685 ,951.1854951185496 ,1048.8145048814504 ,1149.2329149232914 ,1252.4407252440726 ,1350.0697350069736 ,1450.4881450488147])
        csvl_btag_y = np.array([0.5634517766497461,0.7461928934010151,0.7851099830795262,0.7411167512690354,0.6548223350253806,0.6074450084602367,0.5769881556683587,0.4923857868020304,0.4416243654822335,0.43147208121827396,0.3874788494077833,0.3671742808798645,0.34179357021996604,0.34686971235194586,0.3164128595600677])
        ax2_jet.scatter(official_btag_x, official_btag_y, s=80, edgecolors=pt_colors["CSVM"], linewidth=1., facecolors='none', marker=(4,0), label="Official CSVM")
        ax2_jet.scatter(hbb_btag_x, hbb_btag_y, s=80, edgecolors=pt_colors["CSVT"], linewidth=1., facecolors='none', marker=(4,0), label="bbHbb CSVT")
        ax2_jet.scatter(csvl_btag_x, csvl_btag_y, s=80, edgecolors=pt_colors["CSVL"], linewidth=1., facecolors='none', marker=(4,0), label="High mass bb CSVL")

        if jet_index == 0:
            ax2_jet.set_xlabel("Leading $p_T$ [GeV]")
        else:
            ax2_jet.set_xlabel("Subleading $p_T$ [GeV]")
        ax1_jet.set_ylabel("Events")
        ax2_jet.set_ylabel("Offline CSV Efficiency")
        ax2_jet.legend()
        ax1_jet.set_xlim(0., 1200.)
        ax2_jet.set_xlim(0., 1200.)
        ax2_jet.set_ylim(0., 1.)
        
        
        

Hist('hist_jet_pt_CSVT')
Hist('hist_jet_pt_CSVM')
Hist('hist_jet_pt_CSVL')
Hist('hist_jet_pt_NoBTag')
Hist('hist_jet_pt_CSVT')
Hist('hist_jet_pt_CSVM')
Hist('hist_jet_pt_CSVL')
Hist('hist_jet_pt_NoBTag')
Hist('hist_jet_pt_CSVT')
Hist('hist_jet_pt_CSVM')
Hist('hist_jet_pt_CSVL')
Hist('hist_jet_pt_NoBTag')
Hist('hist_jet_pt_CSVT')
Hist('hist_jet_pt_CSVM')
Hist('hist_jet_pt_CSVL')
Hist('hist_jet_pt_NoBTag')
